# [**Comparaison de la recherche aléatoire et de la recherche en grille pour l'estimation d'hyperparamètres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_dsia_skl_doc_fr/blob/main/docs/examples/3_model_selection/plot_randomized_search.ipynb)<br/>([_Comparing randomized search and grid search for hyperparameter estimation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html))

Cette comparaison porte sur la recherche aléatoire et la recherche en grille pour l'optimisation des hyperparamètres d'une SVM linéaire avec un entraînement SGD. Tous les paramètres qui influencent l'apprentissage sont recherchés simultanément (à l'exception du nombre d'estimateurs, qui pose un compromis temps/qualité).

La recherche aléatoire et la recherche en grille explorent exactement le même espace de paramètres. Les résultats en termes de paramètres sont assez similaires, tandis que le temps d'exécution de la recherche aléatoire est nettement plus court.

Les performances peuvent être légèrement moins bonnes pour la recherche aléatoire, et cela est probablement dû à un effet de bruit qui ne se reproduirait pas sur un jeu de test indépendant.

Notez qu'en pratique, on ne rechercherait pas simultanément autant de paramètres différents en utilisant la recherche en grille, mais on choisirait uniquement ceux jugés les plus importants.

In [1]:
from time import time

import numpy as np
import scipy.stats as stats

from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# get some data
X, y = load_digits(return_X_y=True, n_class=3)

# build a classifier
clf = SGDClassifier(loss="hinge", penalty="elasticnet", fit_intercept=True)


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results["rank_test_score"] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print(
                "Mean validation score: {0:.3f} (std: {1:.3f})".format(
                    results["mean_test_score"][candidate],
                    results["std_test_score"][candidate],
                )
            )
            print("Parameters: {0}".format(results["params"][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {
    "average": [True, False],
    "l1_ratio": stats.uniform(0, 1),
    "alpha": stats.loguniform(1e-2, 1e0),
}

# run randomized search
n_iter_search = 15
random_search = RandomizedSearchCV(
    clf, param_distributions=param_dist, n_iter=n_iter_search
)

start = time()
random_search.fit(X, y)
print(
    "RandomizedSearchCV took %.2f seconds for %d candidates parameter settings."
    % ((time() - start), n_iter_search)
)
report(random_search.cv_results_)

# use a full grid over all parameters
param_grid = {
    "average": [True, False],
    "l1_ratio": np.linspace(0, 1, num=10),
    "alpha": np.power(10, np.arange(-2, 1, dtype=float)),
}

# run grid search
grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(X, y)

print(
    "GridSearchCV took %.2f seconds for %d candidate parameter settings."
    % (time() - start, len(grid_search.cv_results_["params"]))
)
report(grid_search.cv_results_)

RandomizedSearchCV took 0.79 seconds for 15 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.989 (std: 0.007)
Parameters: {'alpha': 0.11758035119948389, 'average': False, 'l1_ratio': 0.027179371205144465}

Model with rank: 2
Mean validation score: 0.987 (std: 0.009)
Parameters: {'alpha': 0.013634959481665643, 'average': False, 'l1_ratio': 0.46225302671113155}

Model with rank: 3
Mean validation score: 0.983 (std: 0.025)
Parameters: {'alpha': 0.026877529337749846, 'average': False, 'l1_ratio': 0.2439247267575847}

GridSearchCV took 3.98 seconds for 60 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.993 (std: 0.007)
Parameters: {'alpha': 0.01, 'average': False, 'l1_ratio': 0.4444444444444444}

Model with rank: 2
Mean validation score: 0.991 (std: 0.008)
Parameters: {'alpha': 0.1, 'average': False, 'l1_ratio': 0.2222222222222222}

Model with rank: 3
Mean validation score: 0.991 (std: 0.008)
Parameters: {'alpha': 0.01, 'average': True, 'l